In [1]:
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

In [2]:
#Listings, including full descriptions and average review score
#Reviews, including unique id for each reviewer and detailed comments
#Calendar, including listing id and the price and availability for that day

In [3]:
names = ['calendar', 'listings', 'reviews']
dataframes = {}

for name in names:
    # Define the file path
    file_path = 'data/' + name + '.csv.gz'
    # Use gzip.open to decompress the file and then read it with Pandas
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        data = pd.read_csv(file)

    dataframes[name] = data

In [15]:
listings = dataframes['listings']

## Features som burde fjernes

#### listing_url, scrape_id, last_scraped, source, host_url, host_name, host_thumbnail_url, host_picture_url, license (noen steder må man ha lisens, kunder ser ikke dette), calender_last_scraped, calendar_updated (kun NaN), name, picture_url,



In [16]:
listings = listings.drop(columns=['listing_url', 'scrape_id', 'last_scraped', 'source', 'host_url', 
    'host_name', 'host_thumbnail_url', 'host_picture_url', 'license', 'calendar_last_scraped', 
    'calendar_updated', 'name', 'picture_url'])

## Features som krever tekstsøk 
#### description, neighborhood_overview, host_about




In [17]:
listings = listings.drop(columns=["description", "neighborhood_overview", "host_about"])

## Features som burde fjernes pga mange NaN values:
#### Neighbourhood (kan erstattes av clustering med lat/lon), bathrooms (NaN imputation muligheter med bathroom text), bedrooms (NaN imputation muligheter?)

In [18]:
listings = listings.drop(columns=["neighbourhood", "bathrooms", "bedrooms"])

### Fjerner alle max, min nights features utenom max_nights og min_nights. Ikke strengt talt ubrukelige features, men vil utvilsomt ha høy korrelasjon med max_nights og min_nights, og usikker på hvor mye ekstra forklaringsevne de tilfører problemet. 

### Beholder max og min nights ntm (next twelve months), fordi dette blir noe litt annet, tror dette er lengste og korteste planlagte/booket opphold neste tolv månedene.

In [21]:
listings = listings.drop(columns=["minimum_minimum_nights", "maximum_minimum_nights", 
        "minimum_maximum_nights", "maximum_maximum_nights"])

In [22]:
listings.columns

Index(['id', 'host_id', 'host_since', 'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scor

In [24]:
# Calculate the percentage of NaN values in each column
nan_percentage = (listings.isna().mean() * 100).round(2)

# Convert the nan_percentage Series to a DataFrame for formatting
nan_percentage_df = nan_percentage.reset_index()
nan_percentage_df.columns = ['Column', 'NaN Percentage']

# Print the nan_percentage as a nicely formatted table
print(tabulate(nan_percentage_df, headers='keys', tablefmt='pretty'))

+----+----------------------------------------------+----------------+
|    |                    Column                    | NaN Percentage |
+----+----------------------------------------------+----------------+
| 0  |                      id                      |      0.0       |
| 1  |                   host_id                    |      0.0       |
| 2  |                  host_since                  |      0.01      |
| 3  |                host_location                 |     21.34      |
| 4  |              host_response_time              |     33.96      |
| 5  |              host_response_rate              |     33.96      |
| 6  |             host_acceptance_rate             |     30.69      |
| 7  |              host_is_superhost               |      0.66      |
| 8  |              host_neighbourhood              |     19.98      |
| 9  |             host_listings_count              |      0.01      |
| 10 |          host_total_listings_count           |      0.01      |
| 11 |

#### Jeg har først lyst til å utforske NaN values i host response time, host response rate og host acceptance rate. Jeg ønsker primært å avdekke hvorvidt NaN values stammer fra feil i dataen, eller om NaN values representerer mangel på forespørsler på annonsen over annonsens livsstid eller at disse ratene "resettes" dersom man ikke har hatt forespørsler de siste X årene.

#### Et viktig moment er at dette er statistikk på HOST, ikke på annonsen. Det vil si at denne statistikken påvirkes av andre annonser enn den vi ser på.

#### last_review forteller når man sist mottok en anmeldelse fra leietaker. Dersom alle rader med NaN values på acceptance rate, response rate og response time har siste anmeldelse langt tilbake i tid, så kan dette implisere at disse ratene nullstilles etter et gitt tidspunkt (f.eks etter 2019). 

#### Ser derimot nedenfor at man finner en rad der siste anmeldelse var i 2023, men likevel har NaN values på alle tre rater. Det vil si at det finnes NaN values som ikke kan forklares, og må behandles som missing values.

In [35]:
listings["last_review"] = pd.to_datetime(listings["last_review"])

latest_nan_date_id = listings[listings['host_acceptance_rate'].isna()]['last_review'].idxmax()
listings.loc[5605]

id                                                                                       27620424
host_id                                                                                 112963302
host_since                                                                             2017-01-23
host_location                                                                        New York, NY
host_response_time                                                                            NaN
host_response_rate                                                                            NaN
host_acceptance_rate                                                                          NaN
host_is_superhost                                                                               f
host_neighbourhood                                                                            NaN
host_listings_count                                                                           1.0
host_total_listings_

#### Nå ønsker jeg å se om det er en sammenheng i NaN values mellom de tre ratene, spesifikt acceptance rate vs. response rate. Finner rader der man har NaN på acceptance rate men faktiske verdier for response. Ser også at reviews er NaN, som kan bety at slike rader inntreffer når hosten får en forespørsel men ikke aksepterer den.

In [60]:
acceptance_nan_response_notna = listings[listings['host_acceptance_rate'].isna() 
        & listings['host_response_rate'].notna()]

acceptance_nan_response_notna.iloc[4]

id                                                                                       48093831
host_id                                                                                 385609575
host_since                                                                             2021-01-22
host_location                                                                        New York, NY
host_response_time                                                                 within an hour
host_response_rate                                                                           100%
host_acceptance_rate                                                                          NaN
host_is_superhost                                                                               f
host_neighbourhood                                                                   Far Rockaway
host_listings_count                                                                           1.0
host_total_listings_

#### Sjekker derfor om tilfeller der kun acceptance rate er NaN stammer fra at hosten har svart på forespørsel men sagt nei til leietaker. Ser at det er 191 tilfeller der man har en review + reponse rate, men likevel har NaN value på acceptance rate. Altså virker det som at det er uforklarige missing values her også for acceptance rate.

In [77]:
nan_count = acceptance_nan_response_notna['last_review'].notna().sum()
print("Number of rows in which only acceptance rate is NaN and listing still has review: " + str(nan_count))


Number of rows in which only acceptance rate is NaN and listing still has review: 191


### Sjekker om NaN values i response rate og response time alltid forekommer samtidig

In [71]:
# Condition for rows where 'host_response_time' is NaN and 'host_response_rate' is not NaN
condition_time_nan_rate_notna = listings['host_response_time'].isna() & listings['host_response_rate'].notna()

# Condition for rows where 'host_response_rate' is NaN and 'host_response_time' is not NaN
condition_time_notna_rate_nan = listings['host_response_time'].notna() & listings['host_response_rate'].isna()


count_time_nan_rate_notna = condition_time_nan_rate_notna.sum()
count_time_notna_rate_nan = condition_time_notna_rate_nan.sum()

print(f"Number of rows where 'host_response_time' is NaN and 'host_response_rate' is not NaN: {count_time_nan_rate_notna}")
print(f"Number of rows where 'host_response_time' is not NaN and 'host_response_rate' is NaN: {count_time_notna_rate_nan}")


Number of rows where 'host_response_time' is NaN and 'host_response_rate' is not NaN: 0
Number of rows where 'host_response_time' is not NaN and 'host_response_rate' is NaN: 0


#### Konklusjon: Det finnes missing values for både response og acceptance rate som ikke kan forklares av mangel på opphold den siste perioden eller totalt for annonsens levetid. Videre finnes det rader der response har value mens acceptance er NaN som ikke kan forklares av mangel på faktisk opphold (review som proxy). NaN values i response rate og response time opptrer alltid sammen, som kan antyde at det finnes et mønster i hvordan NaN values oppstår for disse to kolonnene. Jeg tror sannsynligvis at det rett og slett finnes annonser som AirBnB ikke har data på når det gjelder respons og/eller acceptance. Nøyaktig hvorfor de ikke har denne dataen er jeg usikker på.

